# Estimating Method 1: Robinson Difference Estimator
We first introducce a method, which is useful when dealing with additive model with only two additive components.Consider the following the model:
$$
y_i = X'_i\beta + g(z_i)+\epsilon_i \tag{1}
$$
In which $g(z_i)$ is an unknown form. It seems not easy to estimate the $\beta$ since we even do not know the $g$. But Robinson(1989) puts up a Robinson Difference Estimator.Suppose that the $E(\epsilon|x,z)=0$. We notice that
$$
E(y_i|z_i) = E(X'_i |z_i)\beta + g(z_i)+ E(\epsilon |z_i)= E(X'_i |z_i)\beta + g(z_i)+ E_{X_i}[ E(  \epsilon |z_i ,X_i)  ] = E(X'_i |z_i)\beta + g(z_i) \tag{2}
$$
This implies that we get (1)-(2) as
$$
y_i-E(y_i|z_i)= [X'_i-E(X'_i |z_i) ]\beta+\epsilon_i \tag{3}
$$
Then things become easy. We can first estimate $\hat{E}(y_i|z_i)$ and $\hat{E}(X'_i |z_i)$ (via non-parametric methods) and then estimate the $\beta$ from:
$$
y_i-\hat{E}(y_i|z_i)= [X'_i-\hat{E}(X'_i |z_i) ]\beta+\epsilon_i \tag{4}
$$
Finally we can get the non-parametric for $g(z_i)$ 
$$
\hat{g}(z_i)= \hat{E}(y_i|z_i)-\hat{E}(x_i|z_i)'\hat{\beta}
$$

# Estimating method 2: Back Fitting
## Linear Case
We start by first considering the following the linear regression model:
$$
E[Y|X]=\sum_{j}^{p}{\beta_j}X_j
$$
Then we can know that 
$$
E[Y|X_k=x]= \beta_k x + E\left(\sum_{j\ne k}{\beta_j X_j} | X_k= x\right)
$$
which implies that 
$$\beta_k x =E\left(Y-\sum_{j\ne k}{\beta_j X_j} | X_k= x\right)$$
Denote 
$$Y-\sum_{j\ne k}{\beta_j X_j}\equiv Y^{(k)} \tag{5}$$
If we know the data of $Y^{(k)}$, we can easily fit $\beta_k$, since this is just single variable linear regression. The $Y^{(k)}$, in turn, relies on $\beta_0,\beta_2,,..\beta_{k-1},\beta_{k+1},...$. This motivates us to apply a recursive method to estimate all the $\beta$. 

- suppose $X_0$ is constant. we first initialize $\beta$
- do:
    - 1.for each $k=1,2,...p$:
        - calculate the data $Y_k$ according to (5)
        - regress $Y_k$ on data $X_k$ and get the estimate $\beta^{new}_{k}$
    - 2.when $dist(\beta_{new},\beta)<\epsilon$, then stop!
    - else: $\beta^{new}$ $\rightarrow$ $\beta$ and repeat from 1
- Finally, $\beta_0= \overline{y}- \sum_{k=1}^{p}{\beta_k \overline{X_k}}$


In [121]:
def BackFittingLinear(y_data,x_data, criterion = 1e-5, max_iter = 100):
    y = y_data 
    x = x_data
    iter = 1
    n = len(y)
    nk = np.shape(x)[1]
    beta = [0 for i in range(nk)]
    beta_new =  beta.copy()
    while (1):
        # for each k from 1 to nk, calculate the updated beta_k 
        for k in range(1,nk):
            # calculate the data for y_k
            y_k = [y[i]-sum(list(map(lambda a,b: a*b, beta,x[i])))+beta[k]*x[i][k] for i in range(n)]
            # now we give a new estimation for the beta_k
            # first, extract out the data of x_k
            x_k = [x[i][k] for i in range(n)]
            # Then, calculate the beta_new[k]
            beta_new[k]= sum(list (map(lambda a,b: a*b, x_k, y_k)))/sum ([x**2 for x in x_k])
        
        error = sum(list (map(lambda a,b: (a-b)**2, beta,beta_new )))
        if (error <criterion or iter >max_iter):
            break
        beta = beta_new.copy()
        iter = iter +1
    
    # finally calculate the beta[0]
    beta[0]= sum(y)/n- sum([sum([x[i][k] for i in range(n)])*beta[k]/n for k in range(1, nk)])
    return beta, error
                

In [125]:
# A test
# input x and y data.
x= np.array([1,0,3,4,9,23,3,7,3,4,23,4,3,42,20]).reshape(5,3)
y =[34,13,2,78,3]
beta, error = BackFittingLinear(y,x)
print('the estimation is', beta)
print('the error is', error)

the estimation is [13.731748515237767, 1.0822177414989085, -0.49657320070944194]
the error is 7.543939753052188e-06


## General Case
One cute thing about backfitting is that it doesn’t actually rely on the model
being linear.Now consider:
$$
E[Y|X]=\alpha+\sum_{j}^{p}f_j(X_j) \tag{6}
$$
And we do not know the functional form of $f$. 
Denote 
$$Y-\alpha-\sum_{j\ne k}{f_j( X_j)}\equiv Y^{(k)} $$
Then theoretically we can do the back fitting method as in the linear case. We also have the problem of identifiability: suppose $f_1,..f_p$ and $\alpha$ satisfy (6), then $f_1-c,..f_p-c$ and $\alpha+pc$ also satisfy (6). This implies that, in order to make them identifiable, we need to add a constraint on $f$. Such constraint can be :
$$\sum_{i}^{n}{f_j(X_{ki})}=0, \forall k=1,,,p$$
which implies that $$\alpha= \overline{y}$$
 <p>We apply the following algorithm: </p>

-  first initialize $f_1,...f_p$
- do:
    - 1.for each $k=1,2,...p$:
        - calculate the data $Y_k$ according to (6)
        - find a $g_k()$ that can fit data $(X_k,Y_k)$.Can apply nonparametric methods or interporlation.
        - $f^{new}_k(.)=g_k(.)-\frac{1}{n}\sum{g_k(X_{ki})}$
    - 2.when $dist(f^{new},f)<\epsilon$, then stop!
    - else: $f^{new}$ $\rightarrow$ $f$ and repeat from 1
- Finally, $\alpha = \overline{y}$

## Reference
https://www.stat.cmu.edu/~ryantibs/advmethods/notes/addmodels.pdf
https://www4.stat.ncsu.edu/~lu/ST7901/lecture%20notes/2019Lect19_GAM.pdf
https://www.stat.cmu.edu/~cshalizi/350/lectures/21/lecture-21.pdf
    